#installing dependencies

# New Section

In [1]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4
Fri May 13 23:14:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                            

In [2]:
!pip install optuna==2.3.0
!pip install transformers==4.2.1
!pip install farasapy
!pip install pyarabic
# !git clone https://github.com/aub-mind/arabert
!wget https://huggingface.co/UBC-NLP/MARBERT/resolve/main/MARBERT_pytorch_verison.tar.gz
!tar -xvf MARBERT_pytorch_verison.tar.gz
!wget https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_train.tsv
!wget https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_test.tsv
!mkdir -p AJGT
!mv UBC_AJGT_final_shuffled_train.tsv ./AJGT/UBC_AJGT_final_shuffled_train.tsv
!mv UBC_AJGT_final_shuffled_test.tsv ./AJGT/UBC_AJGT_final_shuffled_test.tsv
!pip install GPUtil pytorch_pretrained_bert transformers

--2022-05-13 23:14:42--  https://huggingface.co/UBC-NLP/MARBERT/resolve/main/MARBERT_pytorch_verison.tar.gz
Resolving huggingface.co (huggingface.co)... 3.210.158.153, 34.197.58.156, 34.228.99.153, ...
Connecting to huggingface.co (huggingface.co)|3.210.158.153|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/UBC-NLP/MARBERT/85bfec76f38cba4bc2e6cd02a959016de37ba93de4c850a7d175811dce4e8adc?response-content-disposition=attachment%3B%20filename%3D%22MARBERT_pytorch_verison.tar.gz%22 [following]
--2022-05-13 23:14:43--  https://cdn-lfs.huggingface.co/UBC-NLP/MARBERT/85bfec76f38cba4bc2e6cd02a959016de37ba93de4c850a7d175811dce4e8adc?response-content-disposition=attachment%3B%20filename%3D%22MARBERT_pytorch_verison.tar.gz%22
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 18.155.68.98, 18.155.68.128, 18.155.68.94, ...
Connecting to cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)|18.155.68.98|:443... connected.
HTTP reque

In [3]:
!mkdir data
!mkdir train

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘train’: File exists


#Creating training datasets

In [4]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
all_datasets= []

In [5]:
class Dataset:
    def __init__(
        self,
        name,
        train,
        test,
        label_list,
    ):
        self.name = name
        self.train = train
        self.test = test
        self.label_list = label_list

In [6]:
DATA_COLUMN = "Tweets"
LABEL_COLUMN = "HS"

##HARD - Balanced

In [7]:
df_HARD=pd.read_csv("/content/subTask B (HS&NOT_HS).csv")

In [8]:
train_HARD, test_HARD = train_test_split(df_HARD, test_size=0.1, random_state=42, shuffle=True)
label_list_HARD = [0,1]
data_Hard = Dataset("HARD", train_HARD, test_HARD, label_list_HARD)
all_datasets.append(data_Hard)

#Trainer

In [9]:
!git clone https://github.com/aub-mind/arabert.git
from arabert.preprocess import ArabertPreprocessor
from transformers import AutoTokenizer, AutoModel
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample
import logging
import torch
import optuna

fatal: destination path 'arabert' already exists and is not an empty directory.


In [10]:
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

In [11]:
for x in all_datasets:
  print(x.name)

HARD


You can choose which model, and dataset from here along with the max sentence length

In [12]:
dataset_name = 'HARD'
model_name = 'UBC-NLP/MARBERT'
task_name = 'classification'
max_len = 266

In [13]:
for d in all_datasets:
  if d.name==dataset_name:
    selected_dataset = d
    print('Dataset found')
    break

Dataset found


In [14]:
arabert_prep = ArabertPreprocessor(model_name.split("/")[-1])
selected_dataset.train[DATA_COLUMN] = selected_dataset.train[DATA_COLUMN].apply(lambda x:   arabert_prep.preprocess(x))
selected_dataset.test[DATA_COLUMN] = selected_dataset.test[DATA_COLUMN].apply(lambda x:   arabert_prep.preprocess(x))

In [15]:
selected_dataset.test[DATA_COLUMN]

1564      لعنه تلعنه شاكلته يطلع ضغوطه النفسيه بنات ديرته
1550    والله منافسه بينه سليطين افضل نعال حمود زايد و...
4856    امن العقوبه اساء الادب وين رجال الامن مايجري ا...
2879                                               الخيرا
3544                  شوفي الخاص قحبتي ومتعي كسكك الشرموط
                              ...                        
3066                                         الخير سووميه
3809                                    صوت البنت استفزاز
4932    امنه تبقا امك دندرواي متحرش اسراءيل امنه ومستق...
3857                                    ازعل الدنيا لرضاك
2445                  الدروز اكبر خونه للامه العربيه يكفي
Name: Tweets, Length: 594, dtype: object

In [16]:
class BERTDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask, label=self.label_map[self.target[item]])

In [17]:
label_map = { v:index for index, v in enumerate(selected_dataset.label_list) }
print(label_map)
train_dataset = BERTDataset(selected_dataset.train[DATA_COLUMN].to_list(),selected_dataset.train[LABEL_COLUMN].to_list(),model_name,max_len,label_map)
test_dataset = BERTDataset(selected_dataset.test[DATA_COLUMN].to_list(),selected_dataset.test[LABEL_COLUMN].to_list(),model_name,max_len,label_map)


{0: 0, 1: 1}


In [18]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [19]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  #print(classification_report(p.label_ids,preds))
  #print(confusion_matrix(p.label_ids,preds))
  macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[0,1])
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  return {
      'macro_f1' : macro_f1,
      'macro_f1_pos_neg' : macro_f1_pos_neg,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
  }

you can change the batch size and gradient accumulation from here



#Regular Training

This paert allows you to do a regular training with no hyper parameter optimization

In [20]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate =5e-5
training_args.fp16 = True
training_args.per_device_train_batch_size = 32
training_args.per_device_eval_batch_size = 32
training_args.gradient_accumulation_steps = 5
training_args.num_train_epochs= 4


steps_per_epoch = (len(selected_dataset.train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)
#Warmup_ratio
warmup_ratio = 0.1
training_args.warmup_steps = total_steps*warmup_ratio # or you can set the warmup steps directly 

training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000 #don't want to save any model, there is probably a better way to do this :)
training_args.seed = 30
training_args.disable_tqdm = False
training_args.lr_scheduler_type = 'cosine'

33
132


In [ ]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics = compute_metrics,
)

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Macro F1,Macro F1 Pos Neg,Macro Precision,Macro Recall,Accuracy,Runtime,Samples Per Second
0,No log,0.350740,0.858780,0.858780,0.880952,0.862243,0.860269,3.447700,172.290000
1,No log,0.277987,0.908858,0.908858,0.915858,0.910199,0.909091,3.312500,179.323000
2,No log,0.223426,0.934321,0.934321,0.936062,0.934920,0.934343,3.297200,180.151000
3,No log,0.251960,0.927550,0.927550,0.930596,0.928354,0.927609,3.306100,179.669000


TrainOutput(global_step=132, training_loss=0.20639441230080344, metrics={'train_runtime': 321.5813, 'train_samples_per_second': 0.41, 'total_flos': 5534251821575568, 'epoch': 3.99})

In [23]:
trainer.save_model("HS")

In [24]:
! zip -r '/content/HS.zip' '/content/HS'

  adding: content/HS/ (stored 0%)
  adding: content/HS/config.json (deflated 51%)
  adding: content/HS/training_args.bin (deflated 44%)
  adding: content/HS/pytorch_model.bin (deflated 7%)


In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
import shutil
shutil.move('/content/HS.zip','/content/drive/MyDrive/')

'/content/drive/MyDrive/HS.zip'

In [ ]:
from zipfile import ZipFile
with ZipFile('HS.zip', 'r') as zip:
	# printing all the contents of the zip file
	zip.printdir()

	# extracting all the files
	print('Extracting all the files now...')
	zip.extractall()
	print('Done!')

In [ ]:
import torch 
modell= torch.load('/content/content/rename/pytorch_model.bin')
print(modell)